In [ ]:
!nvidia-smi

Sun Jun  4 10:14:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git 
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 12.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 데이터셋 준비


In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Read the CSV file with the specified encoding
df = pd.read_csv('/content/novel_genre.csv', encoding='cp949')

df['novel'] = df['novel'].apply(lambda x: x[:10000] if len(x) > 10000 else x)

# Create a new DataFrame with the desired column mappings
df = pd.DataFrame({
    'instruction': df['novel'],
    'output': df['genre'],
})

# Create the train and testation datasets
dataset = Dataset.from_pandas(df)

# Create the DatasetDict
data = DatasetDict({'train': dataset})

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 274
    })
})

In [ ]:
# # data
# data = data.map(
#     lambda x: 
#     {'text': f"### 명령어: {x['instruction']}\n\n###맥락: {x['input']}\n\n### 답변: {x['output']}<|endoftext|>" }
#     if x['input'] else 
#     {'text':f"### 명령어: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>"},
# )
# data
data = data.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

data

Map:   0%|          | 0/274 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url', 'text'],
        num_rows: 274
    })
})

## 쪼개진 모델 로드

- 원래는 단일 파일이기도 하지만, 작은 파일(약 1GB)로 쪼개서 개별로 로드한 레포를 쓰면 RAM이 터지지 않습니다.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/KoAlpaca-Polyglot-5.8B"  # safetensors 컨버팅된 레포
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

## 텍스트 데이터만 tokenize

In [ ]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/274 [00:00<?, ? examples/s]

In [ ]:
data['train'][0]['text']

'### 질문: 1화가은은 꽉 닫힌 서재의 문을 한참동안이나 멍하니 서서 바라봤다.들어가면 안 되는 곳을 들어가야 하는 사람처럼 가은은 깊은숨을 내쉬고 나서야 서재 문을 벌컥 열었다.술에 취한 것처럼 그녀의 볼이 붉게 물들었다.갑작스럽게 노크도 없이 들어온 가은의 모습에도 도준은 그녀에게 어떤 시선도 주지 않았다.도준의 그런 태도에 가은은 누군가가 제 목을 세게 쥔 것처럼 숨이 막혀왔다.오늘이 마지막이예요.바짝 마른 가은의 입술이 떨렸다.그런 건 변호사와 얘기하지.도준은 여전히 서류에 시선을 고정한 채 냉랭한 목소리로 그녀를 몰아세웠다.하지만 오늘만큼은 그녀 역시 조금도 물러설 생각이 없었다.삼 년 이란 시간 동안 그의 뒷모습만 바라봤다.결혼식이 끝난 이후 그는 해외지사를 돌면서 집에 들어오는 날보다 들어오지 않는 날이 더 많았다.마지막으로 부탁이 있어요.가은은 마지막 용기를 내며 입술을 움직였다.도준은 대답대신 귀찮다는 듯 살짝 고개를 돌려 가은의 얼굴을 바라봤다.저랑 자요.그녀의 말이 채 끝나기도 전, 도준의 눈썹이 날카롭게 위로 치솟았다.술에 취했으면, 방에서 쉬어.그녀는 술 한 모금도 마시지 않은 맨 정신이었다.오히려 지금이 도준과 함께 마주보는 순간 동안 가장 정신이 또렷했다.다시 도준의 시선이 서류로 가려는 찰나 가은이 천천히 도준의 곁으로 다가왔다.그리고 이내 의자에 앉아있는 도준의 볼에 가볍게 입술을 맞췄다.삼 년이란 무미건조한 결혼 생활을 보상이라도 하려는 듯 그녀가 먼저 도준이 정해놓은 선을 넘었다.가볍게 입술을 맞춘 가은의 눈이 금방이라도 울 것처럼 젖어 들었다.그녀는 서둘러 서재를 빠져 나가려는 듯 도준을 향해 몸을 휙 돌렸다.그때, 도준이 그녀의 손목을 세게 잡아 끌었다.순식간에 가은은 앉아있는 도준의 하체 위에 앉게 되었다.선 넘지 말라고 했을 텐데.냉철하면서도 매혹적인 목소리가 가은의 심장에 꽂혔다.하지만 그녀는 더는 그에게 미안하다는 말을 하고 싶지 않았다.그와 결혼 생활을 하는 내내 입버릇처럼 그에게 뱉었던 말을.마지막 순간까지 

PEFT를 통해 `prepare_model_for_kbit_training`로 Low bit 학습을 준비

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["query_key_value"], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 3670016 || all params: 3070156800 || trainable%: 0.11953838970048696


In [ ]:
!nvidia-smi

Sun Jun  4 10:26:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    28W /  70W |   5481MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 학습

In [ ]:
max_length = 2048  # 모델의 입력 크기

trimmed_data = []
for example in data["train"]:
    input_ids = example["input_ids"]
    attention_mask = example["attention_mask"]

    # 입력 시퀀스를 모델의 입력 크기에 맞게 자름
    trimmed_input_ids = input_ids[:max_length]
    trimmed_attention_mask = attention_mask[:max_length]

    trimmed_example = {
        "input_ids": trimmed_input_ids,
        "attention_mask": trimmed_attention_mask
    }
    trimmed_data.append(trimmed_example)

# 크기 조정된 데이터셋 사용
data["train"] = trimmed_data

In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        max_steps=50, ## 초소량만 학습: 50 step만 학습.
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,4.491200
20,4.037100
30,3.591800
40,3.234000
50,3.071800


TrainOutput(global_step=50, training_loss=3.6851878356933594, metrics={'train_runtime': 601.284, 'train_samples_per_second': 0.166, 'train_steps_per_second': 0.083, 'total_flos': 3621211078656000.0, 'train_loss': 3.6851878356933594, 'epoch': 0.36})

In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:", 
            return_tensors='pt', 
            return_token_type_ids=False
        ), 
        max_new_tokens=1000,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

## Test

In [ ]:
gen('소설을 한 줄 창작해줘')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 소설을 한 줄 창작해줘

### 답변: 소설은 짧은 이야기 형식으로 구성됩니다. 이야기의 요약은 대략 3문장 정도로 짧게 작성합니다. 하지만 이 때, 소설의 시점은 보통 2인칭으로 나타나고, 시점은 자유롭게 사용할 수 있습니다. 따라서, 제 3의 시점으로 화성시와 우주를 배경으로 하는 소설을 쓴다면, 1인칭 주인공이 나올 것입니다. <|endoftext|>


In [ ]:
gen('로맨스 소설의 내용을 한 줄 적어줘')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 로맨스 소설의 내용을 한 줄 적어줘

### 답변: 로맨스 소설의 내용을 한 줄 적어드리겠습니다. 

 로맨스 소설에는 보통 젊은 남자와 여자의 사랑 이야기가 담겨있다. 아주 평범한 짝사랑 이야기일 수도 있고, 어릴 때 사랑했던 첫사랑을 어른이 되어 다시 사랑하는 이야기일 수도 있다. 이는 그들의 추억과 경험을 발판으로 삼으며 그들의 사랑 이야기를 조명한다. 

 로맨스 소설은 일반적인 소설과는 다르게 여러 가지 다른 시각적인 요소보다는 인물 간의 감정과 심리 묘사 중심으로 이야기가 전개된다. 따라서 로맨스 소설의 내용은 주인공의 내면과 감정에 집중된다. 이러한 특징 때문인지 로맨스 소설은 더욱 감성적이고, 더욱 디테일한 이야기를 담고 있다.  

 마치 봄바람처럼 마음을 간질이기 때문에 로맨스 소설이라고 하는 것이다. 하지만, 많은 로맨스 소설이 모두 사랑에 대한 조언이나 충고를 담고 있는 것은 아니다. 로맨스 소설은 주인공들의 사랑 이야기를 담지만, 그들의 인간적인 고민과 감정들을 풀어낸다. 이를 통해 사랑을 하지만, 표현하기 어려운 진짜 사랑이 무엇인지를 그리고, 사랑하지만, 마음대로 되지 않는 상황과 인물의 내면을 묘사함으로써 독자들의 마음을 움직인다. <|endoftext|>


In [ ]:
gen('미스터리 소설의 내용을 한 줄 창작해서 적어줘')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 미스터리 소설의 내용을 한 줄 창작해서 적어줘

### 답변: 미스터리 소설의 내용을 한 줄 창작해서 작성하라고 명을 받았습니다. 이 일은 일어나지 않았습니다. 몇 개월 동안 일을 하지 않았고, 이제는 그 일에 대해서 후회하는 사람들의 시선에 띄게 되는 일을 하고 있어서 적은 답은 ‘재미있는 이야기’입니다. 이것에 대해서는 독자들이 스스로 상상하시는 재미가 있는 이야기를 써주시면 됩니다. 독자들에게는 자유로운 묘사를 부탁드립니다.<|endoftext|>


In [ ]:
gen('로맨스 소설의 내용을 한 줄 적어줘')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 로맨스 소설의 내용을 한 줄 적어줘

### 답변: 로맨스 소설의 내용 요약: 
남녀 간의 로맨스와 사랑 이야기를 다룬 이야기로, 언제나 해피엔딩을 맞이하며, 주로 여성이 주인공이고 로맨스 소설의 내용은 대부분 다음과 같습니다. 1. 남녀 간의 로맨스와 사랑 이야기. 2. 해피엔딩. 3. 여자 작가의 로맨스 소설. 4. 남자 작가의 로맨스 소설. 5. 누구나 작가가 쓰는 로맨스 소설. 6. 여자의 로맨스 소설. 7. 사랑과 인간사이의 로맨스.<|endoftext|>


In [ ]:
gen('작가가 되어서 라이트노벨 소설의 내용을 한 줄 적어줘')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 작가가 되어서 라이트노벨 소설의 내용을 한 줄 적어줘

### 답변: 1. 사람에 대해 잘 몰랐던 그들이지만 이제는 사람만큼 사랑스러운 것이 없다고 생각한다. 2. 인생에서 그의 이름이 유명하지 않았지만, 작은 마을 사람들과의 인터뷰를 하며 자신의 인간관계를 넓힐 수 있었다. 3. 사람과의 관계라는 것은 일이나 업적을 보고 얻는 것이 아니라, 자신의 인간관계를 쌓으면서 넓어지는 것이다. 좋은 관계는 자신이 사랑하는 것만큼 얻을 수 있는 것이다.<|endoftext|>


In [ ]:
gen('작가가 되어서 무협 소설을 창작해서 한 줄 적어줘')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 작가가 되어서 무협 소설을 창작해서 한 줄 적어줘

### 답변: 무협 소설을 창작할 때는 주로 상상을 이용하여 작성합니다. 무협 소설의 설정은 기본적으로 공간 설정, 배경 설정, 캐릭터 설정과 같은 다양한 요소들을 포함해야 하며, 실제 무협 소설에서도 이러한 요소들을 적용시킬 수 있습니다. 무협 소설의 주요 재미는 전투, 수련, 격투, 여행 등이지만, 무협 소설의 설정과 재미는 캐릭터와 스토리와 같은 요소들을 가지고 만들어 낼 수 있습니다. 무협 소설을 쓰기 위해서는 우선 무협 만화나 소설 등 여러 가지 컨텐츠에서 무협과 관련된 설정을 참고하여 쓰는 것이 첫걸음으로, 이후에는 설정을 참고하여 캐릭터와 스토리를 만들어 내는 것이 중요합니다. <|endoftext|>
